In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb

In [3]:
import gc

In [4]:
gc.collect()

0

In [4]:
train = pd.read_csv('C:\Users\ZhangYue\Desktop\\app\\application_train.csv')
test = pd.read_csv('C:\Users\ZhangYue\Desktop\\app\\application_test.csv')

print 'train size', train.shape
print 'test size', test.shape

train size (307511, 122)
test size (48744, 121)


In [7]:
train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500,406597.5,24700.5,...,0,0,0,0,0,0,0,0,0,1
1,100003,0,Cash loans,F,N,N,0,270000,1293502.5,35698.5,...,0,0,0,0,0,0,0,0,0,0
2,100004,0,Revolving loans,M,Y,Y,0,67500,135000.0,6750.0,...,0,0,0,0,0,0,0,0,0,0
3,100006,0,Cash loans,F,N,Y,0,135000,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500,513000.0,21865.5,...,0,0,0,0,0,0,0,0,0,0


#### Concat data and delete columns with too many nulls

In [5]:
train_y = train['TARGET']
dat = pd.concat([train.drop('TARGET', axis=1), test])
dat.shape

(356255, 121)

In [7]:
total = dat.shape[0]
col_null_perc = []
for col in dat.columns:
    perc = sum(dat[col].isnull())/float(total)
    print col, ':', perc
    col_null_perc.append(perc)

SK_ID_CURR : 0.0
NAME_CONTRACT_TYPE : 0.0
CODE_GENDER : 0.0
FLAG_OWN_CAR : 0.0
FLAG_OWN_REALTY : 0.0
CNT_CHILDREN : 0.0
AMT_INCOME_TOTAL : 0.0
AMT_CREDIT : 0.0
AMT_ANNUITY : 0.00010105121331630434
AMT_GOODS_PRICE : 0.0007803399250536834
NAME_TYPE_SUITE : 0.0061837728593282905
NAME_INCOME_TYPE : 0.0
NAME_EDUCATION_TYPE : 0.0
NAME_FAMILY_STATUS : 0.0
NAME_HOUSING_TYPE : 0.0
REGION_POPULATION_RELATIVE : 0.0
DAYS_BIRTH : 0.0
DAYS_EMPLOYED : 0.0
DAYS_REGISTRATION : 0.0
DAYS_ID_PUBLISH : 0.0
OWN_CAR_AGE : 0.660316346437243
FLAG_MOBIL : 0.0
FLAG_EMP_PHONE : 0.0
FLAG_WORK_PHONE : 0.0
FLAG_CONT_MOBILE : 0.0
FLAG_PHONE : 0.0
FLAG_EMAIL : 0.0
OCCUPATION_TYPE : 0.3143703246270228
CNT_FAM_MEMBERS : 5.613956295350241e-06
REGION_RATING_CLIENT : 0.0
REGION_RATING_CLIENT_W_CITY : 0.0
WEEKDAY_APPR_PROCESS_START : 0.0
HOUR_APPR_PROCESS_START : 0.0
REG_REGION_NOT_LIVE_REGION : 0.0
REG_REGION_NOT_WORK_REGION : 0.0
LIVE_REGION_NOT_WORK_REGION : 0.0
REG_CITY_NOT_LIVE_CITY : 0.0
REG_CITY_NOT_WORK_CITY : 0.0
L

In [8]:
dat.drop(np.squeeze(dat.columns[[p>.4 for p in col_null_perc]]), axis=1, inplace=True)
dat.shape

(356255, 72)

In [79]:
dat.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,Cash loans,M,N,Y,0,202500,406597.5,24700.5,351000,...,0,0,0,0,0,0,0,0,0,1
1,100003,Cash loans,F,N,N,0,270000,1293502.5,35698.5,1129500,...,0,0,0,0,0,0,0,0,0,0
2,100004,Revolving loans,M,Y,Y,0,67500,135000.0,6750.0,135000,...,0,0,0,0,0,0,0,0,0,0
3,100006,Cash loans,F,N,Y,0,135000,312682.5,29686.5,297000,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,Cash loans,M,N,Y,0,121500,513000.0,21865.5,513000,...,0,0,0,0,0,0,0,0,0,0


In [34]:
train_y.value_counts()

0    282686
1     24825
Name: TARGET, dtype: int64

#### Modelling

##### Label encoder nominal features

In [9]:
cat_features_idx = []
cols = dat.columns[1:]
for i in range(len(cols)):
    col = cols[i]
    if dat[col].dtypes == 'object':
        cat_features_idx.append(i)
        dat[col] = dat[col].astype('category').cat.codes

In [34]:
dat.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,0,1,0,1,0,202500,406597.5,24700.5,351000,...,0,0,0,0,0,0,0,0,0,1
1,100003,0,0,0,0,0,270000,1293502.5,35698.5,1129500,...,0,0,0,0,0,0,0,0,0,0
2,100004,1,1,1,1,0,67500,135000.0,6750.0,135000,...,0,0,0,0,0,0,0,0,0,0
3,100006,0,0,0,1,0,135000,312682.5,29686.5,297000,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,1,0,1,0,121500,513000.0,21865.5,513000,...,0,0,0,0,0,0,0,0,0,0


In [35]:
gc.collect()

203

In [4]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [11]:
train_X = dat[:307511]
train_X.drop('SK_ID_CURR', axis=1, inplace=True)
print train_X.shape
print train_y.shape

(307511, 71)
(307511L,)


C:\Users\ZhangYue\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [12]:
test = dat[307511:]

test_id = test['SK_ID_CURR']
test.drop('SK_ID_CURR', axis=1, inplace=True)
print test.shape

(48744, 71)


C:\Users\ZhangYue\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.3)

In [18]:
train_data = lgb.Dataset(X_train, label=y_train, feature_name='auto', 
                                  categorical_feature=cat_features_idx)

test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

param = {'num_trees': 4000, 'objective':'binary', 'metric': 'auc'}
bst = lgb.train(param, train_data, valid_sets=[test_data], early_stopping_rounds=100)

[1]	valid_0's auc: 0.70566
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.709326
[3]	valid_0's auc: 0.709891
[4]	valid_0's auc: 0.713331
[5]	valid_0's auc: 0.712052
[6]	valid_0's auc: 0.713956
[7]	valid_0's auc: 0.715181
[8]	valid_0's auc: 0.714948
[9]	valid_0's auc: 0.715507
[10]	valid_0's auc: 0.717034
[11]	valid_0's auc: 0.718613
[12]	valid_0's auc: 0.71976
[13]	valid_0's auc: 0.720987
[14]	valid_0's auc: 0.721411
[15]	valid_0's auc: 0.721794
[16]	valid_0's auc: 0.722144
[17]	valid_0's auc: 0.722718
[18]	valid_0's auc: 0.72357
[19]	valid_0's auc: 0.724061
[20]	valid_0's auc: 0.724564
[21]	valid_0's auc: 0.725087
[22]	valid_0's auc: 0.725785
[23]	valid_0's auc: 0.726674
[24]	valid_0's auc: 0.727152
[25]	valid_0's auc: 0.72784
[26]	valid_0's auc: 0.728867
[27]	valid_0's auc: 0.729381
[28]	valid_0's auc: 0.730006
[29]	valid_0's auc: 0.730855
[30]	valid_0's auc: 0.731937
[31]	valid_0's auc: 0.732522
[32]	valid_0's auc: 0.733118
[33]	valid_0's auc: 0.

In [97]:
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_features)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

param = {'num_trees': 4000, 'objective':'binary', 'metric': 'auc'}
bst = lgb.train(param, train_data, valid_sets=[test_data], early_stopping_rounds=100)

[1]	valid_0's auc: 0.704212
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.708533
[3]	valid_0's auc: 0.711679
[4]	valid_0's auc: 0.711506
[5]	valid_0's auc: 0.714213
[6]	valid_0's auc: 0.715773
[7]	valid_0's auc: 0.716816
[8]	valid_0's auc: 0.716977
[9]	valid_0's auc: 0.718099
[10]	valid_0's auc: 0.718724
[11]	valid_0's auc: 0.719213
[12]	valid_0's auc: 0.719914
[13]	valid_0's auc: 0.721421
[14]	valid_0's auc: 0.722394
[15]	valid_0's auc: 0.722867
[16]	valid_0's auc: 0.723747
[17]	valid_0's auc: 0.724255
[18]	valid_0's auc: 0.724702
[19]	valid_0's auc: 0.725308
[20]	valid_0's auc: 0.726071
[21]	valid_0's auc: 0.726875
[22]	valid_0's auc: 0.727349
[23]	valid_0's auc: 0.728224
[24]	valid_0's auc: 0.729257
[25]	valid_0's auc: 0.729778
[26]	valid_0's auc: 0.730754
[27]	valid_0's auc: 0.731434
[28]	valid_0's auc: 0.732164
[29]	valid_0's auc: 0.732753
[30]	valid_0's auc: 0.733535
[31]	valid_0's auc: 0.7344
[32]	valid_0's auc: 0.735112
[33]	valid_0's auc: 

In [98]:
param = {'num_trees': 2000, 'objective':'binary', 'metric': 'auc'}

tr_D = lgb.Dataset(train_X, label=train_y, categorical_feature=cat_features)
clf = lgb.train(param, tr_D)

In [102]:
y_pred = clf.predict(test)

result = pd.DataFrame({'SK_ID_CURR': test_id, 'TARGET':y_pred})
result.to_csv('baseline_lgb_2.csv', index=False)

In [44]:
from lightgbm import LGBMClassifier

In [45]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.3)

clf = LGBMClassifier(
            n_estimators=4000,
            learning_rate=0.03,
            num_leaves=30,
            colsample_bytree=.8,
            subsample=.9,
            max_depth=7,
            reg_alpha=.1,
            reg_lambda=.1,
            min_split_gain=.01,
            min_child_weight=2,
            silent=-1,
            verbose=-1,
        )
        
clf.fit(X_train, y_train, 
        eval_set= [(X_test, y_test)], 
        eval_metric='auc', verbose=100, early_stopping_rounds=100  #30
       )

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.738811
[200]	valid_0's auc: 0.750536
[300]	valid_0's auc: 0.754431
[400]	valid_0's auc: 0.755708
[500]	valid_0's auc: 0.756349
[600]	valid_0's auc: 0.75677
[700]	valid_0's auc: 0.756961
Early stopping, best iteration is:
[673]	valid_0's auc: 0.757077


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
        learning_rate=0.03, max_depth=7, min_child_samples=20,
        min_child_weight=2, min_split_gain=0.01, n_estimators=4000,
        n_jobs=-1, num_leaves=30, objective=None, random_state=None,
        reg_alpha=0.1, reg_lambda=0.1, silent=-1, subsample=0.9,
        subsample_for_bin=200000, subsample_freq=1, verbose=-1)

In [49]:
y_pred = clf.predict_proba(test)

result = pd.DataFrame({'SK_ID_CURR': test_id, 'TARGET': np.squeeze(y_pred[:, 1])})
result.head()

,SK_ID_CURR,TARGET
0,100001,0.039116
1,100005,0.178387
2,100013,0.017567
3,100028,0.051725
4,100038,0.158463


In [50]:
result.to_csv('baseline_lgb.csv', index=False)

In [5]:
df = pd.read_csv('D:\pyCharm\Creadit_Risk\dat_all.csv')
print df.shape

(356255, 598)


In [6]:
gc.collect()

0

#### Label Encoder

In [7]:
for col in df.drop('TARGET', axis=1).columns:
    if (col != 'SK_ID_CURR') & (df[col].dtypes == 'object'):
        print col
        df[col] = df[col].astype('category').cat.codes

CODE_GENDER
EMERGENCYSTATE_MODE
FLAG_OWN_CAR
FLAG_OWN_REALTY
FONDKAPREMONT_MODE
HOUSETYPE_MODE
NAME_CONTRACT_TYPE
NAME_EDUCATION_TYPE
NAME_FAMILY_STATUS
NAME_HOUSING_TYPE
NAME_INCOME_TYPE
NAME_TYPE_SUITE
OCCUPATION_TYPE
ORGANIZATION_TYPE
WALLSMATERIAL_MODE
WEEKDAY_APPR_PROCESS_START


In [8]:
train = df[:307511]
test = df[307511:]

In [9]:
del df
gc.collect()

0

In [ ]:
train_X = train.drop('TARGET', axis=1)
train_y = train['TARGET']

test = test.drop('TARGET', axis=1)

In [13]:
cat_feats = ['CODE_GENDER',
            'EMERGENCYSTATE_MODE',
            'FLAG_OWN_CAR',
            'FLAG_OWN_REALTY',
            'FONDKAPREMONT_MODE',
            'HOUSETYPE_MODE',
            'NAME_CONTRACT_TYPE',
            'NAME_EDUCATION_TYPE',
            'NAME_FAMILY_STATUS',
            'NAME_HOUSING_TYPE',
            'NAME_INCOME_TYPE',
            'NAME_TYPE_SUITE',
            'OCCUPATION_TYPE',
            'ORGANIZATION_TYPE',
            'WALLSMATERIAL_MODE',
            'WEEKDAY_APPR_PROCESS_START']

In [15]:
param = {'n_estimators': 10000,
        'learning_rate': 0.02,
        'num_leaves': 34,
        'colsample_bytree': 0.9497036,
        'subsample': 0.8715623,
        'subsample_freq': 1,
        'max_depth': 8,
        'reg_alpha': 0.041545473,
        'reg_lambda': 0.0735294,
        'min_split_gain': 0.0222415,
        'min_child_weight': 60,
        'metric': 'auc'}

In [25]:
total = train_X.shape[0]
col_null_perc = []
for col in train_X.columns:
    perc = sum(train_X[col].isnull())/float(total)
    print col, ':', perc
    col_null_perc.append(perc)

AMT_ANNUITY : 3.9022994299390916e-05
AMT_CREDIT : 0.0
AMT_GOODS_PRICE : 0.0009040327012692228
AMT_INCOME_TOTAL : 0.0
AMT_REQ_CREDIT_BUREAU_DAY : 0.13501630835970097
AMT_REQ_CREDIT_BUREAU_HOUR : 0.13501630835970097
AMT_REQ_CREDIT_BUREAU_MON : 0.13501630835970097
AMT_REQ_CREDIT_BUREAU_QRT : 0.13501630835970097
AMT_REQ_CREDIT_BUREAU_WEEK : 0.13501630835970097
AMT_REQ_CREDIT_BUREAU_YEAR : 0.13501630835970097
APARTMENTS_AVG : 0.5074972927797705
APARTMENTS_MEDI : 0.5074972927797705
APARTMENTS_MODE : 0.5074972927797705
BASEMENTAREA_AVG : 0.5851595552679416
BASEMENTAREA_MEDI : 0.5851595552679416
BASEMENTAREA_MODE : 0.5851595552679416
CNT_CHILDREN : 0.0
CNT_FAM_MEMBERS : 6.503832383231819e-06
CODE_GENDER : 0.0
COMMONAREA_AVG : 0.6987229725115525
COMMONAREA_MEDI : 0.6987229725115525
COMMONAREA_MODE : 0.6987229725115525
DAYS_BIRTH : 0.0
DAYS_EMPLOYED : 0.0
DAYS_ID_PUBLISH : 0.0
DAYS_LAST_PHONE_CHANGE : 3.2519161916159095e-06
DAYS_REGISTRATION : 0.0
DEF_30_CNT_SOCIAL_CIRCLE : 0.0033202064316398437

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.3)

train_data = lgb.Dataset(X_train, label=y_train, feature_name='auto', 
                                  categorical_feature=cat_feats)

test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

bst = lgb.train(param, train_data, valid_sets=[test_data], early_stopping_rounds=100)

C:\Users\ZhangYue\AppData\Local\Continuum\Anaconda2\lib\site-packages\lightgbm\engine.py:99: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\ZhangYue\AppData\Local\Continuum\Anaconda2\lib\site-packages\lightgbm\basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')



[354]	valid_0's auc: 0.77164
[355]	valid_0's auc: 0.771659
[356]	valid_0's auc: 0.771696
[357]	valid_0's auc: 0.771734
[358]	valid_0's auc: 0.771796
[359]	valid_0's auc: 0.771804
[360]	valid_0's auc: 0.771871
[361]	valid_0's auc: 0.771902
[362]	valid_0's auc: 0.771942
[363]	valid_0's auc: 0.771958
[364]	valid_0's auc: 0.77197
[365]	valid_0's auc: 0.772025
[366]	valid_0's auc: 0.772035
[367]	valid_0's auc: 0.772061
[368]	valid_0's auc: 0.772079
[369]	valid_0's auc: 0.772101
[370]	valid_0's auc: 0.772181
[371]	valid_0's auc: 0.772218
[372]	valid_0's auc: 0.772261
[373]	valid_0's auc: 0.772291
[374]	valid_0's auc: 0.772293
[375]	valid_0's auc: 0.772304
[376]	valid_0's auc: 0.772337
[377]	valid_0's auc: 0.772384
[378]	valid_0's auc: 0.772384
[379]	valid_0's auc: 0.772432
[380]	valid_0's auc: 0.772419
[381]	valid_0's auc: 0.772425
[382]	valid_0's auc: 0.772437
[383]	valid_0's auc: 0.772472
[384]	valid_0's auc: 0.772557
[385]	valid_0's auc: 0.7726
[386]	valid_0's auc: 0.772629
[387]	valid_0